**Train an autoencoder (very similar to autoencoder.ipynb) on several types of histograms and study the combined prediction**

Note: this is quite long and complicated script that is quite flexible in the method of training and testing. It is not so much a fixed and fully worked out tutorial, but rather a development script. In fact, many of the results I showed in "recent" DQM/DC general meetings were obtained with this script.

In [ ]:
### imports

# external modules
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import importlib

# local modules
sys.path.append('../utils')
import csv_utils as csvu
import json_utils as jsonu
import dataframe_utils as dfu
import hist_utils as hu
import autoencoder_utils as aeu
import plot_utils as pu
import generate_data_utils as gdu
import refruns_utils as rru
importlib.reload(csvu)
importlib.reload(jsonu)
importlib.reload(dfu)
importlib.reload(hu)
importlib.reload(aeu)
importlib.reload(pu)
importlib.reload(gdu)
importlib.reload(rru)
sys.path.append('../src')
sys.path.append('../src/classifiers')
sys.path.append('../src/cloudfitters')
import HistStruct
importlib.reload(HistStruct)
import AutoEncoder
importlib.reload(AutoEncoder)
import SeminormalFitter
import GaussianKdeFitter
import HyperRectangleFitter
importlib.reload(SeminormalFitter)
importlib.reload(GaussianKdeFitter)
importlib.reload(HyperRectangleFitter)

In [ ]:
### define run properties
# in this cell all major run properties are going to be set,
# e.g. what runs to train on and what runs to test on

# define a list of good 'reference' runs (found by eye)
# should be replaced at some point by the reference runs defined by the DQM/DC team.
goodrunsls = {'2017':
              {
                "297056":[[-1]],
                "297177":[[-1]],
                "301449":[[-1]],
              },
              '2018': # needs to be re-checked, not guaranteed to be fully correct or representative.
             {  "315267":[[-1]] 
             }}

# define core test set of clearly bad runs (found by eye)
badrunsls = {'2017':
                {
                "297287":[[-1]],
                "297288":[[-1]],
                "297289":[[-1]],
                "299316":[[-1]],
                "299324":[[-1]],
                "299326":[[-1]],
                "301086":[[88,126]] # only bad for size_PXDisk_+1 -> maybe do not use for now (unclear what are real anomalies)
                },
            '2018': # needs to be re-checked, not guaranteed to be fully correct or representative.
                {
                #"317479":[[-1]],
                "317480":[[-1]],
                "317481":[[-1]],
                "317482":[[-1]],
                #"319847":[[1,35]]
            }}

# set year to use
year = '2017'

# set histogram names to use 
histnames = [
            'chargeInner_PXLayer_2',
             'chargeInner_PXLayer_3',
             'charge_PXDisk_+1','charge_PXDisk_+2','charge_PXDisk_+3',
             'size_PXLayer_1','size_PXLayer_2',
             'size_PXLayer_3'
            ]

# set whether to train globally or locally
training_mode = 'global'

if training_mode == 'global':
    runsls_training = None # use none to not add a mask for training (can e.g. use DCS-bit on mask)
    runsls_good = None # use none to not add a mask for good runs (can e.g. use templates)
    runsls_bad = badrunsls[year] # predefined bad runs
    print('selected runs/lumisections for training: all')
    
elif training_mode == 'local':
    # train locally on a small set of runs
    # - either on n runs preceding a chosen application run,
    # - or on the run associated as reference to the chosen application run.
    
    # select application run
    available_runs = dfu.get_runs( dfu.select_dcson( csvu.read_csv('../data/DF'+year+'_'+histnames[0]+'.csv') ) )
    run_application = 299316
    run_application_index = available_runs.index(run_application)
    # select training set
    usereference = False
    if usereference:
        run_reference = rru.get_reference_run( run_application, jsonfile='../utils/json_allRunsRefRuns.json' )
        if run_reference<0:
            raise Exception('no valid reference run has been defined for run {}'.format(run_application))
        runsls_training = jsonu.tuplelist_to_jsondict([(run_reference,[-1])])
    else:
        ntraining = 5
        offset = 0 # normal case: offset = 0 (just use 5 previous runs)
        runsls_training = jsonu.tuplelist_to_jsondict([(el,[-1]) for el in available_runs[run_application_index-ntraining-offset:run_application_index-offset]])
    #runsls_bad = badrunsls[year]
    #runsls_good = jsonu.tuplelist_to_jsondict([(run_application,[-1])])
    runsls_bad = jsonu.tuplelist_to_jsondict([(run_application,[-1])])
    runsls_good = runsls_training
    print('selected runs/lumisections for training: ')
    print(runsls_training)
    print('selected runs/lumisections as good test set:')
    print(runsls_good)
    print('selected runs/lumisections as bad test set:')
    print(runsls_bad)

In [ ]:
### read the data based on the configuration defined above

readnew = True
save = False

if readnew:
    
    # add the histograms
    histstruct = HistStruct.HistStruct()
    # loop over the histogram types to take into account
    for histname in histnames:
        print('adding {}...'.format(histname))
        # read the histograms from the csv file
        filename = '../data/DF'+year+'_'+histname+'.csv'
        df = csvu.read_csv( filename )
        # in case of local training, we can remove most of the histograms
        if( runsls_training is not None and runsls_good is not None and runsls_bad is not None ):
            runsls_total = {k: v for d in (runsls_training, runsls_good, runsls_bad) for k, v in d.items()}
            df = dfu.select_runsls( df, runsls_total )
        histstruct.add_dataframe( df )
    print('found {} histograms'.format(len(histstruct.runnbs)))
    
    # add masks
    histstruct.add_dcsonjson_mask( 'dcson' )
    histstruct.add_goldenjson_mask('golden' )
    histstruct.add_highstat_mask( 'highstat' )
    histstruct.add_stat_mask( 'lowstat', max_entries_to_bins_ratio=100 )
    if runsls_training is not None: histstruct.add_json_mask( 'training', runsls_training )
    if runsls_good is not None: histstruct.add_json_mask( 'good', runsls_good )
    nbadruns = 0
    if runsls_bad is not None:
        histstruct.add_json_mask( 'bad', runsls_bad )
        # special case for bad runs: add a mask per run (different bad runs have different characteristics)
        nbadruns = len(runsls_bad.keys())
        for i,badrun in enumerate(runsls_bad.keys()):
            histstruct.add_json_mask( 'bad{}'.format(i), {badrun:runsls_bad[badrun]} )
    
    if save:
        histstruct.save( 'test.pkl' )
        
if not readnew:
    
    histstruct = HistStruct.HistStruct.load( 'test.pkl' )
    nbadruns = len([name for name in list(histstruct.masks.keys()) if 'bad' in name])
    
print('created a histstruct with the following properties:')
print('- number of histogram types: {}'.format(len(histstruct.histnames)))
print('- number of lumisections: {}'.format(len(histstruct.lsnbs)))
print('- masks: {}'.format(list(histstruct.masks.keys())))

In [ ]:
### plot the training and/or test sets
# especially useful if running in local mode to check if training set is relevant for targeted application run,
# and if the application run contains anomalies

skipthiscell = True

if( training_mode=='local' and not skipthiscell ):
    
    # training and application runs
    histstruct.plot_histograms( masknames=[['dcson','highstat','training'],['dcson','highstat','good']],
                                labellist = ['training','testing'],
                                colorlist = ['blue','green']
                              )
    
    # application run and bad test runs
    histstruct.plot_histograms( masknames=[['dcson','highstat','good'],['dcson','highstat','bad']],
                                labellist = ['good','bad'],
                                colorlist = ['green','red']
                              )
    
if( training_mode=='global' and not skipthiscell ):
    
    # bad test runs
    for i in [0,1,2,3,4,5,6]:
        histstruct.plot_histograms( masknames=[['dcson','highstat','good'],['dcson','highstat','bad{}'.format(i)]],
                                labellist = ['typical good histograms','bad'],
                                colorlist = ['blue','red'],
                                transparencylist = [0.01,1.]
                                  )

In [ ]:
### extend the training set using artificial data

extendtraining = False

if extendtraining:
    histstruct.exthistograms['training'] = {}
    for histname in histstruct.histnames:
        # option 1: start from 'training' mask
        hists = histstruct.get_histograms( histname=histname, masknames=['dcson','highstat','training'] )
        # option 2: start from averages of DCS-on data
        #hists = hu.averagehists( 
        #            histstruct.get_histograms( histname=histname, masknames=['dcson','highstat'] ), 
        #            1000 )
        print('generating artificial training data for '+histname)
        histstruct.exthistograms['training'][histname] = gdu.upsample_hist_set(hists, 5e4, figname='f' )
        print(' -> generated {} histograms'.format(len(histstruct.exthistograms['training'][histname])))

In [ ]:
### define and train an autoencoder for each element

from tensorflow.keras.models import load_model

trainnew = False
save = False
modelloc = '../models/autoencoders_global_training_dcson_highstat_v20210622'
modelbasename = ''

if trainnew:
    for histname in histstruct.histnames:
        # choose training set
        hists = histstruct.get_histograms( histname=histname, masknames=['dcson','highstat'] )
        if extendtraining: hists = histstruct.exthistograms['training'][histname]
        print('size of training set: {}'.format(hists.shape))
        # choose whether to save the model
        modelname = modelbasename+'_'+histname+'.h5'
        modelname = os.path.join(modelloc,modelname)
        if not save: modelname = '' # empty string means do not save models
        nepochs = 40 # manual number of epochs
        model = aeu.train_simple_autoencoder(hists,nepochs=nepochs,modelname=modelname,
                                            batch_size=2000
                                            )
        classifier = AutoEncoder.AutoEncoder( model=model )
        histstruct.add_classifier(histname,classifier)
    
else:
    from autoencoder_utils import mseTop10
    for histname in histstruct.histnames:
        print('loading model for {}'.format(histname))
        modelname = modelbasename+'_'+histname+'.h5'
        modelname = os.path.join(modelloc,modelname)
        model = load_model(modelname,custom_objects={'mseTop10':mseTop10})
        classifier = AutoEncoder.AutoEncoder( model=model )
        histstruct.add_classifier(histname,classifier)

In [ ]:
### evaluate the models on all histograms in the (non-extended) histstruct

mse_train = []
mse_good = []
mse_bad = []
for i in range(nbadruns): mse_bad.append([])
for histname in histstruct.histnames:
    print('evaluating model for '+histname)
    histstruct.evaluate_classifier(histname)
    # get mse for training set
    if 'training' in histstruct.masks.keys():
        thismse = histstruct.get_scores( histname=histname, masknames=['dcson','highstat', 'training'] )
    else:
        thismse = histstruct.get_scores( histname=histname, masknames=['dcson','highstat'] )
    mse_train.append( thismse )
    # get mse for good set
    if 'good' in histstruct.masks.keys(): 
        thismse = histstruct.get_scores( histname=histname, masknames=['dcson','highstat','good'] )
    else:
        hists_good = hu.averagehists( histstruct.get_histograms( histname=histname, masknames=['dcson','highstat']), 1000 )
        thismse = histstruct.classifiers[histname].evaluate( hists_good )
    mse_good.append( thismse )
    # get mse for bad sets
    for i in range(nbadruns):
        thismse = histstruct.get_scores( histname=histname, masknames=['dcson','highstat','bad{}'.format(i)] )
        mse_bad[i].append( thismse )
        
# transform to arrays with correct shape
mse_train = np.array(mse_train)
mse_train = np.transpose(mse_train)
print('found mse array for training set of following shape: {}'.format(mse_train.shape))
mse_good = np.array(mse_good)
mse_good = np.transpose(mse_good)
print('found mse array for good set of following shape: {}'.format(mse_good.shape))
for i in range(nbadruns):
    mse_bad[i] = np.array(mse_bad[i])
    mse_bad[i] = np.transpose(mse_bad[i])
    print('found mse array for bad set of following shape: {}'.format(mse_bad[i].shape))

In [ ]:
### plot the multidemensional mse and fit a distribution

dimslist = []
fitfunclist = []
nhisttypes = len(histstruct.histnames)
for i in range(0,nhisttypes-1):
    for j in range(i+1,nhisttypes):
        dimslist.append((i,j))

plt.close('all')
(npoints,ndims) = mse_train.shape

# settings for GaussianKdeFitter
scott_bw = npoints**(-1./(ndims+4))
bw_method = 20*scott_bw
# settings for HyperRectangleFitter
quantiles = ([0.00062,0.0006,0.00015,0.00015,
             0.0003,0.0003,0.00053,0.00065])

for dims in dimslist:
    thismse = mse_train[:,dims]
    if training_mode=='global': 
        fitfunc = SeminormalFitter.SeminormalFitter(thismse)
        #fitfunc = HyperRectangleFitter.HyperRectangleFitter(thismse, 
        #                                                    [quantiles[dims[0]],quantiles[dims[1]]],
        #                                                    'up')
    else: fitfunc = GaussianKdeFitter.GaussianKdeFitter(thismse,bw_method=bw_method)
    pu.plot_fit_2d(thismse, fitfunc=fitfunc, logprob=True, clipprob=True,
                    onlycontour=False, xlims=30, ylims=30, 
                    onlypositive=True, transparency=0.5,
                    xaxtitle=histstruct.histnames[dims[0]], 
                    yaxtitle=histstruct.histnames[dims[1]],
                    title='density fit of lumisection MSE')
    #plt.close('all') # release plot memory
    fitfunclist.append(fitfunc)
    
if training_mode=='global': 
    fitfunc = SeminormalFitter.SeminormalFitter(mse_train)
    #fitfunc = HyperRectangleFitter.HyperRectangleFitter(mse_train, quantiles, 'up')
else: fitfunc = GaussianKdeFitter.GaussianKdeFitter(mse_train,bw_method=bw_method)

In [ ]:
### extend the test set using artificial data generation and evaluate the model on the extended test set

skipthiscell = False # to prevent running this cell by accident

if not skipthiscell:
    
    histstruct.exthistograms['good'] = {}
    for i in range(nbadruns): histstruct.exthistograms['bad{}'.format(i)] = {}
    for histname in histstruct.histnames:
        print('generating data for '+histname)
        if 'good' in histstruct.masks.keys():
            goodhists = histstruct.get_histograms( histname=histname,masknames=['dcson','highstat','good'] )
        else:
            goodhists = hu.averagehists( histstruct.get_histograms( histname=histname, masknames=['dcson','highstat'] ), 15 )
        histstruct.exthistograms['good'][histname] = gdu.upsample_hist_set( goodhists,
                                                    figname='',ntarget=nbadruns*5e3,fourierstdfactor=20.)
        # alternative: copy original good set (e.g. for using resampled bad but original good)
        #histstruct.exthistograms['good'][name] = goodhists
        for i in range(nbadruns):
            badhists = histstruct.get_histograms( histname=histname,masknames=['dcson','highstat','bad{}'.format(i)] )
            histstruct.exthistograms['bad{}'.format(i)][histname] = gdu.upsample_hist_set(
                badhists,figname='',ntarget=5e3,fourierstdfactor=20.)
            
    mse_good_ext = []
    mse_bad_ext = []
    for i in range(nbadruns): mse_bad_ext.append( [] )

    for histname in histstruct.histnames:
        print('evaluating: '+histname)
        # calculate mse for good test set
        mse = histstruct.classifiers[histname].evaluate( histstruct.exthistograms['good'][histname] )
        mse_good_ext.append(mse)
        for i in range(nbadruns):
            # calculate mse
            mse = histstruct.classifiers[histname].evaluate( histstruct.exthistograms['bad{}'.format(i)][histname] )
            mse_bad_ext[i].append(mse)
    
    # transform to arrays with correct shape
    mse_good_ext = np.array(mse_good_ext)
    mse_good_ext = np.transpose(mse_good_ext)
    print('found mse array for good set of following shape: {}'.format(mse_good_ext.shape))
    for i in range(nbadruns):
        mse_bad_ext[i] = np.array(mse_bad_ext[i])
        mse_bad_ext[i] = np.transpose(mse_bad_ext[i])
        print('found mse array for bad set of following shape: {}'.format(mse_bad_ext[i].shape))

In [ ]:
### (re-)define the test set

use_ext = False

mse_good_eval = mse_good
mse_bad_eval = mse_bad
if use_ext:
    mse_good_eval = mse_good_ext
    mse_bad_eval = mse_bad_ext
    
# subselect only specific bad sets for quick checking
# note: not very clean, e.g. watch out with using nbadsets after this cell...
mse_bad_eval = [mse_bad_eval[i] for i in [4]]

In [ ]:
### make a new plot of probability contours and overlay data points
### (only 2D projections!)  

doplot = True

if doplot:
    plt.close('all')
    colorlist = ['red','lightcoral','firebrick','chocolate','fuchsia','orange','purple']
    #colorlist = ['red']*nbadsets
    if len(colorlist)<len(mse_bad_eval):
        raise Exception('ERROR: need more colors...')

    for dims,partialfitfunc in zip(dimslist,fitfunclist):
        fig,ax = pu.plot_fit_2d(mse_train[:,dims], fitfunc=partialfitfunc, logprob=True, clipprob=True, 
                    onlycontour=True, xlims=30, ylims=30, 
                    onlypositive=True, transparency=0.5,
                    xaxtitle=histstruct.histnames[dims[0]], 
                    yaxtitle=histstruct.histnames[dims[1]],
                    title='density fit of lumisection MSE')
        for j in range(len(mse_bad_eval)): ax.plot(mse_bad_eval[j][:,dims[0]],mse_bad_eval[j][:,dims[1]],
                                               '.',color=colorlist[j],markersize=4)
        ax.plot(mse_good_eval[:,dims[0]],mse_good_eval[:,dims[1]],'.',color='blue',markersize=4)
    

# get the minimum log probability of histograms in good set
print('--- good lumesections ---')
logprob_good = np.log(fitfunc.pdf(mse_good_eval))
print('length of log prob array: '+str(len(logprob_good)))
print('minimum of log prob: '+str(np.min(logprob_good)))
#print(sorted(logprob_good))
print('--- bad lumisections ---')
logprob_bad_parts = [np.log(fitfunc.pdf(mse_bad_eval[j])) for j in range(len(mse_bad_eval))]
#for lp in logprob_bad_parts: print(str(sorted(lp))+'\n\n')
logprob_bad = np.concatenate(tuple(logprob_bad_parts))
print('length of log prob array: '+str(len(logprob_bad)))
print('maximum of log prob: '+str(np.max(logprob_bad)))
#print(sorted(logprob_good))
#print(sorted(logprob_bad))
#print(logprob_bad)

In [ ]:
### make a roc curve based on the test results above
# note: smaller logprob = less probable = more outlier = more anomalous
# so if anomalies are signal and good histograms are background, -logprob is a suitable score definition,
# since everything above a certain threshold will be considered signal and below it background.

labels_good = np.zeros(len(logprob_good)) # background: label = 0
labels_bad = np.ones(len(logprob_bad)) # signal: label = 1

labels = np.concatenate(tuple([labels_good,labels_bad]))
scores = np.concatenate(tuple([-logprob_good,-logprob_bad]))
scores = aeu.clip_scores( scores )

pu.plot_score_dist(scores, labels, siglabel='anomalous', sigcolor='r', 
                   bcklabel='good', bckcolor='g', 
                   nbins=200, normalize=True,
                   xaxtitle='negative logarithmic probability',
                   yaxtitle='number of lumisections (normalized)')

auc = aeu.get_roc(scores, labels, mode='geom', doprint=False)

logprob_threshold = 60 # everything below this logprob will be considered signal (i.e. anomalous)
aeu.get_confusion_matrix(scores,labels,-logprob_threshold)

In [ ]:
### investigate particular lumisections

mode = 'ls'
run = 297100
ls = 233 # ignored if mode is 'run'
masknames = ['dcson','highstat'] # if mode is 'ls', used for mse plotting
plot_mse = False # ignored if mode is 'run'

#print(histstruct.runnbs[:10])
#print(histstruct.lsnbs[:10])

# define reference histograms
refhists = {}
for histname in histstruct.histnames:
    if( 'good' in histstruct.masks.keys() ): 
        #refhists[histname] = histstruct.get_histograms(masknames=['highstat','dcson','good'])
        refhists[histname] = hu.averagehists( histstruct.get_histograms( histname=histname, masknames=['highstat','dcson','good']), 15 )
    else: 
        refhists[histname] = hu.averagehists( histstruct.get_histograms( histname=histname, masknames=['dcson','highstat'] ), 15 )

if mode=='ls':
    # plot this particular run/ls
    _ = histstruct.plot_ls( run, ls, recohist='auto', refhists=refhists )
    msepoint = histstruct.get_scores_ls( run, ls )
    msepointarray = np.array([msepoint[histname] for histname in histstruct.histnames])
    logprob = np.log(fitfunc.pdf(np.array([msepointarray])))
    print('-------------')
    print('MSE values:')
    for histname in histstruct.histnames: print('{} : {}'.format(histname,msepoint[histname]))
    print('-------------')
    print('logprob: '+str(logprob))
    # plot mse distribution
    if plot_mse:
        for dim,histname in enumerate(histnames):
            mses = histstruct.get_scores( histname=histname, masknames=masknames )
            #mses = mse_good_eval[:,dim]
            nmses = len(mses)
            labels = np.zeros(nmses)
            mses = np.concatenate((mses,np.ones(int(nmses/15))*msepoint[histname]))
            labels = np.concatenate((labels,np.ones(int(nmses/15))))
            pu.plot_score_dist( mses, labels, nbins=200, normalize=False,
                        siglabel='this lumisection', bcklabel='all lumisections',
                        title=histname )

if mode=='run':
    # plot given run
    runnbs = histstruct.get_runnbs( masknames=masknames )
    lsnbs = histstruct.get_lsnbs( masknames=masknames )
    runsel = np.where(runnbs==run)
    lsnbs = lsnbs[runsel]
    print('plotting {} lumisections...'.format(len(lsnbs)))
    for lsnb in lsnbs:
        fig,ax = histstruct.plot_ls(run, lsnb, recohist='auto', refhists=refhists )
        plt.show()
        msepoint = histstruct.get_scores_ls( run, lsnb )
        msepointarray = np.array([msepoint[histname] for histname in histstruct.histnames])
        logprob = np.log(fitfunc.pdf(np.array([msepointarray])))
        print('-------------')
        print('MSE values:')
        for histname in histstruct.histnames: print('{} : {}'.format(histname,msepoint[histname]))
        print('-------------')
        print('logprob: '+str(logprob))

In [ ]:
### investigate how the method performs on the golden/custom test set

#evaljson = jsonu.loadjson('utils/json_pixel_good_201201.json')
#histstruct.add_json_mask( 'pixelgood', evaljson )
masks_eval = ['golden', 'lowstat']
lsnbs_eval = histstruct.get_lsnbs( masknames=masks_eval )
runnbs_eval = histstruct.get_runnbs( masknames=masks_eval )
mse_eval_dict = histstruct.get_scores( masknames=masks_eval )
mse_eval = []
for histname in histstruct.histnames:
    mse_eval.append( mse_eval_dict[histname] )
mse_eval = np.array(mse_eval)
mse_eval = np.transpose(mse_eval)
print('found mse array for evaluation set of following shape: {}'.format(mse_eval.shape))
logprob_eval = np.log(fitfunc.pdf(mse_eval))
#print(logprob_eval)

# define reference histograms
refhists = {}
for histname in histstruct.histnames:
    if( 'good' in histstruct.masks.keys() ): 
        #refhists[histname] = histstruct.get_histograms(masknames=['highstat','dcson','good'])
        refhists[histname] = hu.averagehists( histstruct.get_histograms( histname=histname, masknames=['highstat','dcson','good']), 15 )
    else: 
        refhists[histname] = hu.averagehists( histstruct.get_histograms( histname=histname, masknames=['dcson','highstat'] ), 15 )

def get_runsls_inrange(logprob, runnbs, lsnbs, logprob_up=None, logprob_down=None):
    # get a list of tuples of (run number, ls number) corresponding to ls with log probability within a given range
    # - logprob, runnbs and lsnbs are equally long 1D arrays
    # - logprob_up and logprob_down are upper and lower thresholds
    #     if both are not None, the lumisections with logprob between the boundaries are returned
    #     if logprob_up is None, the lumisections with logprob > logprob_down are returned
    #     if logprob_down is None, the lumisections with logprob < logprob_up are returned
    indices = np.array([])
    if logprob_down is None:
        indices = np.nonzero(logprob<logprob_up)[0]
    elif logprob_up is None:
        indices = np.nonzero(logprob>logprob_down)[0]
    else:
        indices = np.nonzero((logprob>logprob_down) & (logprob<logprob_up))[0]
    runsinrange = runnbs[indices]
    lsinrange = lsnbs[indices]
    runslsinrange = []
    for rr,lsr in zip(runsinrange,lsinrange):
        runslsinrange.append((int(rr),int(lsr)))
    return {'indices':indices,'runslsinrange':runslsinrange}

logup = 1000
logdown = None
temp = get_runsls_inrange(logprob_eval, runnbs_eval, lsnbs_eval,
                          logprob_up = logup, logprob_down = logdown)

runslsinrange = temp['runslsinrange']
print('{} out of {} LS are within these boundaries'.format(len(runslsinrange),len(logprob_eval)))

# make plots
nplotsmax = 20
from matplotlib.backends.backend_pdf import PdfPages
pdf = PdfPages('autoencoder_combine_output.pdf')
for i,(runnb,lsnb) in enumerate(runslsinrange):
    if i>=nplotsmax:
        print('maximum number of plots reached')
        break
    print('------------------------')
    fig,axs = histstruct.plot_ls( runnb, lsnb, recohist='auto', refhists=refhists)
    plt.show()
    pdf.savefig(fig)
    msepoint = histstruct.get_scores_ls( runnb, lsnb )
    msepointarray = np.array([msepoint[histname] for histname in histstruct.histnames])
    logprob = np.log(fitfunc.pdf(np.array([msepointarray])))
    print('-------------')
    print('MSE values:')
    for histname in histstruct.histnames: print('{} : {}'.format(histname,msepoint[histname]))
    print('-------------')
    print('logprob: '+str(logprob))
    # only for 2 dimensions: extra contour plot
    #if nhisttypes != 2: continue
    #fig,ax = plt.subplots()
    #contourplot = ax.contourf(x, y, np.log(fitfunc.pdfgrid(pos)))
    #plt.colorbar(contourplot)
    #ax.plot(msepoint[0],msepoint[1],'.k',markersize=10)
    #ax.set_xlim((0.,xlim))
    #ax.set_ylim((0.,ylim))
pdf.close()

In [ ]:
### investigate OMS data: get data

import sys
sys.path.append('../omsapi')
from get_oms_data import get_oms_api,get_oms_data,get_oms_response_attribute
# get the api instance
omsapi = get_oms_api()
# pileup and luminosity in good test set
oms_target_run = 297100
oms_info = get_oms_data( omsapi, 'lumisections', oms_target_run, attributes=['lumisection_number','pileup','delivered_lumi','recorded_lumi'] )
# L1 and HLT rates in good test set
oms_info_l1 = get_oms_data( omsapi, 'l1triggerrates', oms_target_run, extraargs={'group[granularity]':'lumisection'} )
path_filter = {'attribute_name':'path_name','value':'HLT_ZeroBias_v5','operator':'EQ'}
oms_info_hlt = get_oms_data( omsapi, 'hltpathrates', oms_target_run, extraargs={'group[granularity]':'lumisection'}, extrafilters=[path_filter] )

In [ ]:
### investigate OMS data: make plots

importlib.reload(pu)

oms_lsnbs = get_oms_response_attribute(oms_info,'lumisection_number')
oms_dcson = jsonu.isdcson( [oms_target_run]*len(oms_lsnbs), oms_lsnbs )
pileup = get_oms_response_attribute(oms_info,'pileup')
lumi_del = get_oms_response_attribute(oms_info,'delivered_lumi')
lumi_rec = get_oms_response_attribute(oms_info,'recorded_lumi')
l1_rate = get_oms_response_attribute(oms_info_l1,'l1a_physics')
l1_rate = [el['rate'] for el in l1_rate]
l1_lsnbs = get_oms_response_attribute(oms_info_l1,'first_lumisection_number') # can differ from oms_lsnbs since some lumisection numbers can be missing apparently
l1_rate = [(l1_rate[l1_lsnbs.index(i)] if i in l1_lsnbs else None) for i in oms_lsnbs]
hlt_rate = get_oms_response_attribute(oms_info_hlt,'rate')
hlt_lsnbs = get_oms_response_attribute(oms_info_hlt,'first_lumisection_number') # can differ from oms_lsnbs since some lumisection numbers can be missing apparently
hlt_rate = [(hlt_rate[hlt_lsnbs.index(i)] if i in hlt_lsnbs else None) for i in oms_lsnbs]


doslice = False
if doslice:
    cropslice=slice(40,70,None)
    oms_lsnbs = oms_lsnbs[cropslice]
    oms_dcson = oms_dcson[cropslice]
    pileup = pileup[cropslice]
    lumi_rec = lumi_rec[cropslice]
    lumi_del = lumi_del[cropslice]
    l1_rate = l1_rate[cropslice]
    hlt_rate = hlt_rate[cropslice]
    
_ = pu.plot_hists([pileup], colorlist=['b'], labellist=['pileup'], 
              xlims=(oms_lsnbs[0],oms_lsnbs[-1]),
              title='pileup for run {}'.format(oms_target_run), xaxtitle='lumisection number', yaxtitle='pileup',
              bkgcolor=oms_dcson, bkgcmap='cool', bkgrange=(0,1), bkgtitle='DCS bit')
_ = pu.plot_hists([lumi_rec,lumi_del], colorlist=['g','b'], labellist=['recorded luminosity','delivered luminosity'],
              xlims=(oms_lsnbs[0],oms_lsnbs[-1]),
              title='luminosity for run {}'.format(oms_target_run), xaxtitle='lumisection number', yaxtitle='luminosity',
              bkgcolor=oms_dcson,bkgcmap='cool', bkgrange=(0,1), bkgtitle='DCS bit')
_ = pu.plot_hists([l1_rate], colorlist=['b'], labellist=['L1 trigger rate'], 
              xlims=(oms_lsnbs[0],oms_lsnbs[-1]),
              title='L1 trigger rate for run {}'.format(oms_target_run), xaxtitle='lumisection number', yaxtitle='rate',
              bkgcolor=oms_dcson,bkgcmap='cool', bkgrange=(0,1), bkgtitle='DCS bit')
_ = pu.plot_hists([hlt_rate], colorlist=['b'], labellist=['HLT trigger rate'], 
              xlims=(oms_lsnbs[0],oms_lsnbs[-1]),
              title='HLT trigger rate for run {}'.format(oms_target_run), xaxtitle='lumisection number', yaxtitle='rate',
              bkgcolor=oms_dcson,bkgcmap='cool', bkgrange=(0,1), bkgtitle='DCS bit')

#lsnb = 20
#idx_oms = np.where(np.array(oms_lsnbs)==lsnb)[0][0]
#print('average pileup for this lumisection: {}'.format(pileup[idx_oms]))